# 3.1 Tickers
The goal is to solve the problems of the Polygon ticker lists in the introduction. Before we do that we will download the ticker list for all days from Polygon and store them into the map <code>tickers</code>.

In [8]:
###
from polygon.rest import RESTClient
from datetime import datetime, date, time, timedelta
from pytz import timezone
from functools import lru_cache
from times import get_market_dates, first_trading_date_after, last_trading_date_before, first_trading_date_after_equal, last_trading_date_before_equal
import os
import pytz
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import mplfinance as mpf

POLYGON_DATA_PATH = "../data/polygon/"

START_DATE = date(2003, 9, 10)
END_DATE = date(2024, 4, 19)

with open(POLYGON_DATA_PATH + "secret.txt") as f:
    KEY = next(f).strip()

client = RESTClient(api_key=KEY)

In [12]:
download_tickers(END_DATE).to_csv('../data/POLYGON_ACTIVE')

First, I will create a function to download the tickers for a specific date. We also need to download the 'None' type tickers. 

In [11]:
###
def download_tickers(day):
    """Retrieve the ticker list for a specific date

    Args:
        day (Date): the Date for which to download the ticker list

    Returns:
        DataFrame: the ticker list
    """
    
    date_iso = day.isoformat()

    tickers_iterator_active = client.list_tickers(date=date_iso, active=True, market='stocks', limit=1000)
    tickers = pd.DataFrame(tickers_iterator_active)
    tickers = tickers[~tickers['type'].isin(['PFD', 'WARRANT', 'RIGHT', 'BOND', 'ETF', 'ETN', \
        'ETV', 'SP', 'ADRP', 'ADRW', 'ADRR', 'FUND', 'BASKET', 'UNIT', 'LT', 'GDR', 'OTHER', \
            'AGEN', 'EQLK', 'ETS', 'INDEX'])] # Only keep CS, ADRC, NYRS and OS

    tickers.sort_values(by = "ticker", inplace=True)
    tickers.reset_index(inplace=True, drop=True)
    return tickers[['ticker', 'name', 'active', 'delisted_utc', 'last_updated_utc', 'cik', 'composite_figi', 'type']]

Then all ticker lists are downloaded and stored in the <code>raw/tickers/</code> map. But only the one that we need if we already have some.

In [24]:
###
# Get a list of what we already have
files = os.listdir(POLYGON_DATA_PATH + 'raw/tickers')
available_dates = [date.fromisoformat(file.replace(".csv", "")) for file in files]

trading_dates = get_market_dates()
for day in trading_dates:
    # Only download what we do not have
    if day >= START_DATE and day <= END_DATE and day not in available_dates:
        tickers = download_tickers(day)
        tickers.to_csv(POLYGON_DATA_PATH + f"raw/tickers/{day.isoformat()}.csv")

A random ticker list:

In [4]:
pd.read_csv(POLYGON_DATA_PATH + f"raw/tickers/2022-06-09.csv", index_col=0).head(3)

,ticker,name,active,delisted_utc,last_updated_utc,cik,composite_figi,type
0,A,Agilent Technologies Inc.,True,NaN,2022-06-14T00:00:00Z,1090872.0,BBG000C2V3D6,CS
1,AA,"Alcoa, Inc.",False,2016-11-01T00:00:00Z,2016-11-01T00:00:00Z,4281.0,NaN,CS
2,AA,ALCOA INC,False,2016-10-07T00:00:00Z,2016-10-07T00:00:00Z,4281.0,NaN,CS


We observe that the <code>last_updated_utc</code> does not match the date of the ticker list. For example for "A", this date is *after* 2022-06-09. So this value is not point-in-time. Nevertheless, this value is useless for us because we only use the data to determine the delisting date. Neither do we need <code>delisted_utc</code> for the same reason.

Later when we do have data, we will create a new column <code>start_data</code> and <code>end_data</code> which gives the start and end dates from the available data.

# More problems
The amount of None values for the ticker types is problematic. Sometimes, there isn't even a ticker description. We will clean the ticker lists in the following way:
* Remove tickers with no description or no ticker. (What?)
* Assign ticker type 'NONE'. When merging in step 3.5, if the correct ticker type shows up we will use that. Else we will 'manually' take care of it, e.g. by assigning ADRC to all tickers that have 'DEP', 'ADR' and 'ADS' in its name.

After around 2019, this issue was fixed. So we don't have to do this for new ticker lists. 
<!-- * Assign ticker type 'ADR' if the ticker description contains:
    * DEP, ADR, ADS, PLC, n.v., has a suffix with () that contains a non-US country. 
* Assign ticker type 'CS' otherwise.
Btw, there are also a lot of preferred stocks and other stuff.  -->

In [80]:
files = os.listdir(POLYGON_DATA_PATH + 'raw/tickers')
for file in files:
    df = pd.read_csv(POLYGON_DATA_PATH + f'raw/tickers/{file}', index_col=0)
    df = df[~df['name'].isna()] # Remove tickers with no name
    df = df[~df['ticker'].isna()]
    df['type'] = df['type'].fillna('NONE')
    df.reset_index(drop=True).to_csv(POLYGON_DATA_PATH + f'raw/tickers/{file}')

Btw, some ADRs are classified as common stocks...

# 3.2 Building the tickers loop
Now we can finally create our ticker list, which includes all tickers. The process involves looping over all Polygon ticker lists and updating our own one. First some notation: T is our ticker list that we iteratively update using Polygons ticker list. P(i) is the Polygon ticker list from day *i*. 

1. On day 1, our ticker list is the same as the one from Polygon, but with some extra columns. We create a column <code>start_date</code> which is day 1 and <code>end_date</code> with is empty. We are only interested in stocks that were active on that day.
2. For all *i = 2 ... n* days, for the active stocks:
    * **Delistings**: The stocks that are in T but not in P(i) are the stocks that are removed by Polygon (e.g. FB). For these tickers we set the <code>end_date</code> in T to day *i-1*. 
    * **New listings**: The stocks that are in P(i) but not in T are the new listings. We will append the new stock to T and set the start_date to day *i*.
    * **Everything else**: The stocks that are both in P(i) and T are the stocks that 'continue their listings'. We do nothing.

Two tickers are the 'same' if all fields except <code>last_updated_utc</code> or <code>delisted_utc</code> are the same.

For testing, we will start with 2022-06-08 and update to 2022-06-09. Both FB and META should then be included with the correct start and end dates. The start and end date of FB should be 2022-06-08 and the start date of META should be 2022-06-09. The end date of META should be empty.

In [40]:
day_1 = date(2022, 6, 8)
day_2 = date(2022, 6, 9)

our_tickers = pd.read_csv(
    POLYGON_DATA_PATH + f"raw/tickers/{day_1.isoformat()}.csv",
    index_col=0,
)
our_tickers = our_tickers[["ticker", "name", "active", "cik", "composite_figi", "type"]]
our_tickers = our_tickers[our_tickers["active"] == True]
our_tickers.reset_index(inplace=True, drop=True)

our_tickers["start_date"] = day_1
our_tickers["end_date"] = pd.NaT

tickers_day_2 = pd.read_csv(
    POLYGON_DATA_PATH + f"raw/tickers/{day_2.isoformat()}.csv",
    index_col=0,
)
tickers_day_2 = tickers_day_2[["ticker", "name", "active", "cik", "composite_figi", "type"]]
tickers_day_2 = tickers_day_2[tickers_day_2["active"] == True]
tickers_day_2.reset_index(inplace=True, drop=True)

In [41]:
our_tickers.head(2)

,ticker,name,active,cik,composite_figi,type,start_date,end_date
0,A,Agilent Technologies Inc.,True,1090872.0,BBG000C2V3D6,CS,2022-06-08,NaT
1,AA,Alcoa Corporation,True,1675149.0,BBG00B3T3HD3,CS,2022-06-08,NaT


In [42]:
tickers_day_2.head(2)

,ticker,name,active,cik,composite_figi,type
0,A,Agilent Technologies Inc.,True,1090872.0,BBG000C2V3D6,CS
1,AA,Alcoa Corporation,True,1675149.0,BBG00B3T3HD3,CS


Preliminary check for duplicates

In [43]:
# Preliminary check: no duplicates
if our_tickers[["ticker", "name", "active", "cik", "composite_figi", "type"]].duplicated().any():
    raise Exception("There are duplicates!")

if tickers_day_2[["ticker", "name", "active", "cik", "composite_figi", "type"]].duplicated().any():
    raise Exception("There are duplicates!")

We will first get the delisting and new listings. (Nothing has to be done with the kept listings).

In [44]:
# DELISTINGS: Get tickers that are in T but not in P(2). This is actually not straightforward (https://stackoverflow.com/questions/28901683/pandas-get-rows-which-are-not-in-other-dataframe). We need to get the rows in tickers_day_2 that are not in our_tickers. We will use the merge function but specifying indicator=True and use a left merge (tickers_day_2 left merge to our_tickers). What gets returned is a dataframe with the flags "left_only", "right_only" and "both". If the indicator is "left_only", it means that it existed in only in the left DataFrame (our_tickers). This is exactly what we need. 
indicator_delisted = our_tickers[["ticker", "name", "active", "cik", "composite_figi", "type"]].merge(tickers_day_2[["ticker", "name", "active", "cik", "composite_figi", "type"]], on=["ticker", "name", "active", "cik", "composite_figi", "type"], 
                   how='left', indicator=True)
indicator_delisted = indicator_delisted["_merge"] # Only get the indicator

delisted_tickers = our_tickers[indicator_delisted == "left_only"] # Only get the delisted tickers

# NEW LISTINGS: Swap the DataFrames
indicator_new = tickers_day_2[["ticker", "name", "active", "cik", "composite_figi", "type"]].merge(our_tickers[["ticker", "name", "active", "cik", "composite_figi", "type"]], on=["ticker", "name", "active", "cik", "composite_figi", "type"], 
                   how='left', indicator=True)
indicator_new = indicator_new["_merge"]
new_tickers = tickers_day_2[indicator_new == "left_only"]

# KEPT LISTINGS
current_tickers = our_tickers[indicator_delisted == "both"]
# current_tickers = tickers_day_2[indicator_new == "both"] # It does not matter which one we choose

In [45]:
print(len(delisted_tickers))
delisted_tickers.head(2)

8


,ticker,name,active,cik,composite_figi,type,start_date,end_date
1195,CERN,Cerner Corp,True,804753.0,BBG000BFDLV8,CS,2022-06-08,NaT
1589,CUK,Carnival PLC,True,1125259.0,BBG000CKYG71,ADRC,2022-06-08,NaT


In [46]:
print(len(new_tickers))
new_tickers.head(2)

7


,ticker,name,active,cik,composite_figi,type
1588,CUK,Carnival Plc,True,1125259.0,BBG000CKYG71,ADRC
2545,GLAQ,Globis Acquisition Corp. common stock,True,1823383.0,NaN,CS


In [47]:
len(current_tickers)

6520

Then we will process the delistings and listings.

In [48]:
# DELISTINGS: register delisting date and set to inactive.
our_tickers.loc[indicator_delisted == "left_only", "end_date"] = day_1 #Not day_2!
our_tickers.loc[indicator_delisted == "left_only", "active"] = False

our_tickers[our_tickers["ticker"] == "FB"]

,ticker,name,active,cik,composite_figi,type,start_date,end_date
2171,FB,"Meta Platforms, Inc. Class A Common Stock",False,1326801.0,BBG000MM2P62,CS,2022-06-08,2022-06-08


In [49]:
# NEW LISTINGS: append the new tickers and register start date
print(len(our_tickers))
print(len(new_tickers))

our_tickers = pd.concat([our_tickers, new_tickers])
our_tickers.reset_index(inplace=True, drop=True)
our_tickers['start_date'].fillna(value=day_2, inplace=True)

print(len(our_tickers))
our_tickers[our_tickers["ticker"] == "META"]

6528
7
6535


,ticker,name,active,cik,composite_figi,type,start_date,end_date
6532,META,"Meta Platforms, Inc. Class A Common Stock",True,1326801.0,BBG000MM2P62,CS,2022-06-09,NaN


Some final checks and setting <code>end_date</code> for the active listings at END_DATE.

In [50]:
if our_tickers[["ticker", "name", "active", "type", "start_date"]].isnull().values.any():
    raise Exception("There are missing values.")

# After all is done, set the end_date for active stocks to the new day. This is only done after all iterations. 
our_tickers["end_date"].fillna(value=day_2, inplace=True)

The result is correct. FB is included with the correct <code>end_date</code>. Then META starts with the correct <code>start_date</code>.

In [51]:
our_tickers[our_tickers['ticker'].isin(['FB', 'META'])]

,ticker,name,active,cik,composite_figi,type,start_date,end_date
2171,FB,"Meta Platforms, Inc. Class A Common Stock",False,1326801.0,BBG000MM2P62,CS,2022-06-08,2022-06-08
6532,META,"Meta Platforms, Inc. Class A Common Stock",True,1326801.0,BBG000MM2P62,CS,2022-06-09,2022-06-09


Further down the line, I got an error that I wouldn't have gotten if I checked for duplicates among the active tickers given a day. For the raw polygon ticker lists, I assumed that for all active tickers on that day, there were no duplicates. However I was wrong:

In [52]:
our_tickers = pd.read_csv(
    POLYGON_DATA_PATH + f"raw/tickers/2024-03-01.csv",
    index_col=0,
)
our_tickers = our_tickers[our_tickers["active"] == True]
our_tickers.sort_values(['last_updated_utc'])
our_tickers.reset_index(inplace=True, drop=True)

duplicated = our_tickers[our_tickers["ticker"].duplicated(keep=False)]
duplicated[duplicated["ticker"].duplicated(keep=False)]

,ticker,name,active,delisted_utc,last_updated_utc,cik,composite_figi,type
939,CADE,Cadence Bancorporation,True,NaN,2019-01-03T00:00:00Z,1614184.0,BBG00BZSZZD5,CS
940,CADE,Cadence Bank,True,NaN,2024-04-03T00:00:00Z,1299939.0,BBG000D3MJP5,CS
3421,MRT,"MedEquities Realty Trust, Inc.",True,NaN,2019-01-03T00:00:00Z,1616314.0,NaN,CS
3422,MRT,"Marti Technologies, Inc.",True,NaN,2024-04-15T00:00:00Z,1852767.0,BBG00ZXBPJV8,CS


We need to make sure to remove these. We will simply keep the last one as that is the most recent one (we have already sorted on 'last_updated_utc').

In [53]:
indices_to_remove = duplicated["ticker"].duplicated(keep='last')
list(indices_to_remove[indices_to_remove].index)

[939, 3421]

In [54]:
our_tickers.drop(list(indices_to_remove[indices_to_remove].index), inplace=True)
our_tickers.reset_index(drop=True, inplace=True)

# 3.3 The tickers loop
2004-07-08 is an empty ticker list for some reason. Also the ticker lists of 2009-10-29, 2010-03-30 and 2010-03-31 seem incomplete. We will simply copy the ticker list the trading day before to avoid errors.

Putting it all in a loop gives the following code. We save the results to <code>tickers_v1.csv</code>.

In [ ]:
market_days = get_market_dates()

START_DATE = first_trading_date_after_equal(START_DATE)
END_DATE = last_trading_date_before_equal(END_DATE)

for day in market_days:
    if day == START_DATE:
        # At the start, our ticker list is the same as polygon.
        our_tickers = pd.read_csv(
            POLYGON_DATA_PATH + f"raw/tickers/{START_DATE.isoformat()}.csv",
            index_col=0,
            keep_default_na=False,
            na_values=['#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN', '-NaN', '-nan', '1.#IND', '1.#QNAN', '<NA>', 'N/A', 'NULL', 'NaN', 'None', 'n/a', 'nan', 'null']
        ) #There is a stock named 'NA'. We have to avoid pandas treating it as a N/A value.
        our_tickers = our_tickers[["ticker", "name", "active", "cik", "composite_figi", "type"]]
        our_tickers = our_tickers[our_tickers["active"]]

        # Sometimes a cik can be empty. This prevents the merge function from working.
        our_tickers['cik'] = our_tickers['cik'].replace('', np.nan)
        our_tickers['cik'] = our_tickers['cik'].astype(float)

        our_tickers.reset_index(inplace=True, drop=True)
    
        # Initialize tickers_all
        our_tickers["start_date"] = START_DATE
        our_tickers["end_date"] = pd.NaT

    elif day > START_DATE and day <= END_DATE:
        # Get new ticker list to update ours
        tickers_day_i = pd.read_csv(
            POLYGON_DATA_PATH + f"raw/tickers/{day.isoformat()}.csv",
            index_col=0,
            keep_default_na=False,
            na_values=['#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN', '-NaN', '-nan', '1.#IND', '1.#QNAN', '<NA>', 'N/A', 'NULL', 'NaN', 'None', 'n/a', 'nan', 'null']
        )
        # Sometimes a cik can be empty. This prevents the merge function from working.
        tickers_day_i['cik'] = tickers_day_i['cik'].replace('', np.nan)
        tickers_day_i['cik'] = tickers_day_i['cik'].astype(float)

        tickers_day_i.sort_values(['last_updated_utc'], inplace=True)

        tickers_day_i = tickers_day_i[["ticker", "name", "active", "cik", "composite_figi", "type"]]
        tickers_day_i = tickers_day_i[tickers_day_i["active"]]
        tickers_day_i.reset_index(inplace=True, drop=True)

        # Remove duplicates
        duplicated = tickers_day_i[tickers_day_i["ticker"].duplicated(keep=False)]
        indices_to_remove = duplicated["ticker"].duplicated(keep='last')
        tickers_day_i.drop(list(indices_to_remove[indices_to_remove].index), inplace=True)
        tickers_day_i.reset_index(drop=True, inplace=True)

        # Preliminary check: no duplicates
        if our_tickers.duplicated().any():
            raise Exception("There are duplicates!")

        if tickers_day_i.duplicated().any():
            raise Exception("There are duplicates!")

        # DELISTINGS
        indicator_delisted = our_tickers[["ticker", "name", "active", "cik", "composite_figi", "type"]].merge(tickers_day_i[["ticker", "name", "active", "cik", "composite_figi", "type"]], on=["ticker", "name", "active", "cik", "composite_figi", "type"], how='left', indicator=True)

        indicator_delisted['_merge'] = np.where(our_tickers["active"], indicator_delisted['_merge'], "both") # ERROR FIX: If in our ticker list we have already set it inactive, it should not be added to the list of delisted stocks again. By setting _merge to "both" we skip the already inactive stocks.

        indicator_delisted = indicator_delisted["_merge"] # Only get the indicator
        delisted_tickers = our_tickers[indicator_delisted == "left_only"]

        # NEW LISTINGS
        indicator_new = tickers_day_i[["ticker", "name", "active", "cik", "composite_figi", "type"]].merge(our_tickers[["ticker", "name", "active", "cik", "composite_figi", "type"]], on=["ticker", "name", "active", "cik", "composite_figi", "type"], 
                        how='left', indicator=True)
        indicator_new = indicator_new["_merge"]
        new_tickers = tickers_day_i[indicator_new == "left_only"]

        # PROCESS DELISTINGS
        previous_day = market_days[market_days.index(day) - 1] # Getting previous trading day
        our_tickers.loc[indicator_delisted == "left_only", "end_date"] = previous_day
        our_tickers.loc[indicator_delisted == "left_only", "active"] = False
        
        # PROCESS NEW LISTINGS
        our_tickers = pd.concat([our_tickers, new_tickers])
        our_tickers.reset_index(inplace=True, drop=True)
        our_tickers['start_date'].fillna(value=day, inplace=True)

        # Final checks
        if our_tickers[["ticker", "name", "active", "type", "start_date"]].isnull().values.any():
            #null_data = our_tickers[our_tickers[["ticker", "name", "active", "type", "start_date"]].isnull().any(axis=1)]
            raise Exception("There are missing values.")
        
        print(f'{day.isoformat()}: Amount of stocks {len(our_tickers)}')
        
        # Finalize
        if day == END_DATE:
            our_tickers["end_date"].fillna(value=END_DATE, inplace=True)
            our_tickers = our_tickers.sort_values(by=["ticker", "end_date"]).reset_index(drop=True)
            our_tickers[["ticker", "name", "active", "start_date", "end_date", "type", "cik", "composite_figi"]].\
                to_csv("../data/tickers_v1.csv")

We also create a function to retrieve the ticker list.

In [4]:
def get_tickers(v=5):
    """
    Retrieve the ticker list. Default is 5.
    """
    tickers = pd.read_csv(
        f"../data/tickers_v{v}.csv",
        parse_dates=["start_date", "end_date"],
        index_col=0,
        keep_default_na=False,
        na_values=["#N/A","#N/AN/A","#NA","-1.#IND","-1.#QNAN","-NaN","-nan","1.#IND","1.#QNAN","<NA>","N/A","NULL","NaN","None","n/a","nan","null",],
    )
    tickers["start_date"] = pd.to_datetime(tickers["start_date"]).dt.date
    tickers["end_date"] = pd.to_datetime(tickers["end_date"]).dt.date

    # This will only be applied in future notebooks.
    if tickers.columns.isin(["start_data", "end_data"]).any():
        tickers["start_data"] = pd.to_datetime(tickers["start_data"]).dt.date
        tickers["end_data"] = pd.to_datetime(tickers["end_data"]).dt.date

    return tickers

In [129]:
tickers_v1 = get_tickers(1)
tickers_v1[tickers_v1["ticker"] == "FB"]

,ticker,name,active,start_date,end_date,type,cik,composite_figi
16251,FB,FACEBOOK INC CL A COM STK (DE),False,2012-05-18,2016-10-24,NONE,1326801.0,BBG000MM2P62
16252,FB,"Facebook, Inc. Class A",False,2016-10-25,2021-10-29,CS,1326801.0,BBG000MM2P62
16253,FB,"Meta Platforms, Inc. Class A Common Stock",False,2021-11-01,2022-06-08,CS,1326801.0,BBG000MM2P62


In [130]:
tickers_v1[tickers_v1["ticker"] == "META"]

,ticker,name,active,start_date,end_date,type,cik,composite_figi
27993,META,"Meta Platforms, Inc. Class A Common Stock",True,2022-06-09,2024-04-19,CS,1326801.0,BBG000MM2P62


In [131]:
print(len(tickers_v1))

49011


# 3.4 Checks

1. Are SPACs handled correctly? We should expect that when they IPO a company, that they get delisted. Then one day after the delisting the new-born company should be listed. We will take a look at VFS. On 2023-8-15 it was IPO'd by the SPAC named BSAQ. So we should expect the delisting date of BSAQ to be 2023-8-14.


In [132]:
tickers_v1[tickers_v1["ticker"] == "VFS"]

,ticker,name,active,start_date,end_date,type,cik,composite_figi
45916,VFS,VinFast Auto Ltd. Ordinary Shares,True,2023-08-15,2024-04-19,CS,1913510.0,


In [133]:
tickers_v1[tickers_v1["ticker"] == "BSAQ"]

,ticker,name,active,start_date,end_date,type,cik,composite_figi
6338,BSAQ,Black Spade Acquisition Co,False,2021-09-07,2023-08-14,CS,1851908.0,


2. Let's check SVB which went bankrupt and HTZ which went from OTC to listed.

In [134]:
tickers_v1[tickers_v1["ticker"] == "SIVB"]

,ticker,name,active,start_date,end_date,type,cik,composite_figi
40043,SIVB,SILICON VALLEY BANCSHARES,False,2003-09-10,2005-05-27,NONE,719739.0,
40044,SIVB,SVB FINANCIAL GROUP,False,2005-05-31,2005-08-17,NONE,719739.0,BBG000BT0CM2
40045,SIVB,SVB FINANCIAL GROUP,False,2006-01-12,2016-10-24,NONE,719739.0,BBG000BT0CM2
40046,SIVB,SVB Financial Group,False,2016-10-25,2023-03-27,CS,719739.0,BBG000BT0CM2


In [135]:
tickers_v1[tickers_v1["ticker"] == "HTZ"]

,ticker,name,active,start_date,end_date,type,cik,composite_figi
21788,HTZ,HERTZ GLOBAL HOLDINGS INC COM,False,2006-11-16,2016-07-01,CS,1364479.0,
21789,HTZ,"HERTZ GLOBAL HOLDINGS, INC. COM (DE)",False,2016-07-05,2016-10-24,CS,1657853.0,BBG00D5SHJH6
21790,HTZ,"Hertz Global Holdings, Inc.",False,2016-10-25,2020-10-29,CS,1657853.0,BBG00D5SHJH6
21791,HTZ,"Hertz Global Holdings, Inc Common Stock",True,2021-11-09,2024-04-19,CS,1657853.0,BBG011N57109


3. Sometimes tickers are re-used (e.g. META, but since it was an ETF it will not show up in our ticker list). Let's see if that has happened in our ticker list.

In [136]:
duplicated = tickers_v1[tickers_v1["ticker"].duplicated(keep=False)]
print(len(duplicated["ticker"].unique()))
print(len(duplicated))
duplicated.head()

11410
37377


,ticker,name,active,start_date,end_date,type,cik,composite_figi
0,A,"AGILENT TECHNOLOGIES, INC",False,2003-09-10,2004-06-24,NONE,1090872.0,BBG000C2V3D6
1,A,"AGILENT TECHNOLOGIES, INC",False,2004-06-25,2005-11-22,CS,1090872.0,BBG000C2V3D6
2,A,"AGILENT TECHNOLOGIES, INC",False,2005-11-28,2005-11-28,CS,1090872.0,BBG000C2V3D6
3,A,"AGILENT TECHNOLOGIES, INC",False,2005-11-30,2005-11-30,CS,1090872.0,BBG000C2V3D6
4,A,"AGILENT TECHNOLOGIES, INC",False,2005-12-02,2005-12-02,CS,1090872.0,BBG000C2V3D6


We will have some merging to do. However these are the "clean" ones. The next ones are just ridiculous and should not exist in the first place. 

In [137]:
from collections import Counter
print(Counter(duplicated["ticker"].values.tolist()).most_common(5))
duplicated[duplicated["ticker"] == "DGICA"].head(5)
# ???

[('CMS', 240), ('IBN', 240), ('PRE', 233), ('EP', 183), ('CUK', 106)]


,ticker,name,active,start_date,end_date,type,cik,composite_figi
12385,DGICA,DONEGAL GROUP INC. CL-A,False,2004-11-08,2004-11-08,NONE,800457.0,BBG000JQJC22
12386,DGICA,DONEGAL GROUP INC. CL-A,False,2005-01-03,2012-12-13,NONE,800457.0,BBG000JQJC22
12387,DGICA,Donegal Group Inc,False,2016-10-25,2022-08-25,CS,800457.0,BBG000JQJC22
12388,DGICA,Donegal Group Inc,False,2022-08-26,2022-08-26,CS,948046.0,BBG000JQJC22
12389,DGICA,Donegal Group Inc,False,2022-08-29,2022-09-02,CS,800457.0,BBG000JQJC22


In [138]:
duplicated[duplicated["ticker"] == "DISCK"].head(5)

,ticker,name,active,start_date,end_date,type,cik,composite_figi
12615,DISCK,DISCOVERY COMM INC SER C COM STK,False,2008-09-19,2012-12-13,NONE,1024333.0,
12616,DISCK,DISCOVERY COMM INC SER C COM STK,False,2014-02-14,2014-05-01,NONE,1024333.0,
12617,DISCK,DISCOVERY COMM INC SER C COM STK,False,2014-05-05,2014-05-16,NONE,1024333.0,
12618,DISCK,DISCOVERY COMM INC SER C COM STK,False,2014-05-29,2014-05-29,NONE,1024333.0,
12619,DISCK,DISCOVERY COMM INC SER C COM STK,False,2014-06-02,2014-06-16,NONE,1024333.0,


On average there are 3 duplicates for duplicated tickers. When we take a look it seems that it happens a lot that the name/cik/composite_figi gets changed, even though it is the same company and ticker. For example for ZWS the name is "Zurn Water Solutions Corporation" on 2022-07-01 but on the next trading day (4th July was a stock holiday) the name changes to "Zurn Elkay Water Solutions Corporation". 

# 3.5 Merging duplicates
The most straightforward way to merge these duplicates is to see for every duplicate whether the the end_date (1st occurence) and start_date (2nd occurence) are consecutive *trading days*. Now you understand why we made the functions in notebook 2.

In [139]:
tickers_v1.head(3)

,ticker,name,active,start_date,end_date,type,cik,composite_figi
0,A,"AGILENT TECHNOLOGIES, INC",False,2003-09-10,2004-06-24,NONE,1090872.0,BBG000C2V3D6
1,A,"AGILENT TECHNOLOGIES, INC",False,2004-06-25,2005-11-22,CS,1090872.0,BBG000C2V3D6
2,A,"AGILENT TECHNOLOGIES, INC",False,2005-11-28,2005-11-28,CS,1090872.0,BBG000C2V3D6


First we need to get the duplicates.

In [140]:
tickers_v1 = get_tickers(1)
duplicated = tickers_v1[tickers_v1["ticker"].duplicated(keep=False)]
duplicated.tail(3)

,ticker,name,active,start_date,end_date,type,cik,composite_figi
49004,ZYME,Zymeworks Inc.,True,2022-10-24,2024-04-19,CS,1937653.0,BBG019XSYC89
49005,ZYNE,ZYNERBA PHARMACEUTICALS INC COM STK (DE),False,2015-08-05,2016-10-24,NONE,1621443.0,BBG007BBS8B7
49006,ZYNE,"Zynerba Pharmaceuticals, Inc",False,2016-10-25,2023-10-11,CS,1621443.0,BBG007BBS8B7


In [141]:
###
tickers_v1 = get_tickers(1)
market_days = get_market_dates()

duplicated = tickers_v1[tickers_v1["ticker"].duplicated(keep=False)] # Get all duplicated, *including* the original

# Step 1: Get the indices of the rows that should be merged.
indices_duplicated = [] # looks like [['A', {1, 2, 3}], ['A', {4, 5}], ['B', {10, 11, 12, 13}]]
prev_index_and_row = None
prev_is_duplicate_and_back_to_back = False

for index, row in duplicated.iterrows():
    # Get attributes of previous ticker
    if prev_index_and_row is not None:
        prev_index = prev_index_and_row[0]
        prev_row = prev_index_and_row[1]
        prev_ticker = prev_row["ticker"]
        prev_name = prev_row["name"]
        prev_start_date = prev_row["start_date"]
        prev_end_date = prev_row["end_date"]
        prev_cik = prev_row["cik"]
        prev_figi = prev_row["composite_figi"]

    # Get attributes of current ticker
    current_index = index
    current_row = row
    current_ticker = current_row["ticker"]
    current_name = current_row["name"]
    current_start_date = current_row["start_date"]
    current_end_date = current_row["end_date"]
    current_cik = current_row["cik"]
    current_figi = current_row["composite_figi"]
    
    # Skip first index
    if prev_index_and_row is None:
        pass
    # Check if ticker duplicated and back-to-back
    elif prev_ticker == current_ticker and market_days[market_days.index(prev_end_date) + 1] == current_start_date:
        # If the previous was NOT duplicated/back-to-back, we need to create a new entry for the stock
        if prev_is_duplicate_and_back_to_back == False:
            indices_duplicated.append([current_ticker, {prev_index, current_index}])
        # Else the stock already exists in the list. Then simply append the indices.
        else:
            indices_duplicated[-1][-1].add(prev_index)
            indices_duplicated[-1][-1].add(current_index)
        
        # Update flag
        prev_is_duplicate_and_back_to_back = True
    else:
        prev_is_duplicate_and_back_to_back = False

    # Update prev_index_and_row for next iteration
    prev_index_and_row = (current_index, row)

In [142]:
print(len(indices_duplicated))
print(indices_duplicated[:3])

9927
[['A', {0, 1}], ['A', {8, 9, 10, 11, 12}], ['AA', {13, 14, 15, 16, 17, 18}]]


Very rarely, it happens that the same ticker, but not the same company, has duplicates. E.g. if in our ticker list the first 5 rows is the ticker AAA, but the start and end dates are (1, 2), (2, 3), (3, 4), (9, 10), (10, 11), this means that these are two different companies. Then indices_duplicated contains ['AAA', {1, 2, 3, 4}] and ['AAA, {10, 11}]. If it *is* the same company, it managed to get delisted to OTC and revive to get their listing back. However we stated that we are not interested in OTC so this is fine. Or something is wrong with Polygons data.

In [ ]:
# See which tickers are duplicated in indices_duplicated.
tickers_already_checked = set() 
for ticker, indices in indices_duplicated:
    if ticker in tickers_already_checked:
        print(ticker)
    tickers_already_checked.add(ticker)

Now that we have a list of indices of the duplicated tickers, we can merge them together. We do this by looping over <code>indices_duplicated</code> and then changing all duplicated rows to get the correct values. Then we remove the duplicates.

In [144]:
###

# Step 2: Merge duplicated in tickers_all
"""
Which value is assigned:
    name: last
    active: last
    start_date: first
    end_date: last
    type: last (but does not matter as it is always CS or ADRC)
    cik: last value that is not NaN
    compositite_figi: last value that is not NaN
"""
for ticker, indices in indices_duplicated:
    # CAUTION: Make sure that indices is sorted! Else it can happen that end_date is before start_date. I only found this out later. Moral: Always do sanity checks.
    indices = sorted(list(indices))
    ticker_data_in_tickers_v1 = tickers_v1.iloc[indices, :]
    tickers_v1.iloc[indices, tickers_v1.columns.get_loc("name")] = ticker_data_in_tickers_v1["name"].values[-1]
    tickers_v1.iloc[indices, tickers_v1.columns.get_loc("active")] = ticker_data_in_tickers_v1["active"].values[-1]
    tickers_v1.iloc[indices, tickers_v1.columns.get_loc("start_date")] = ticker_data_in_tickers_v1["start_date"].values[0]
    tickers_v1.iloc[indices, tickers_v1.columns.get_loc("end_date")] = ticker_data_in_tickers_v1["end_date"].values[-1]
    tickers_v1.iloc[indices, tickers_v1.columns.get_loc("type")] = ticker_data_in_tickers_v1["type"].values[-1]
    tickers_v1.iloc[indices, tickers_v1.columns.get_loc("cik")] = ticker_data_in_tickers_v1["cik"].ffill().values[-1]
    tickers_v1.iloc[indices, tickers_v1.columns.get_loc("composite_figi")] = ticker_data_in_tickers_v1["composite_figi"].ffill().values[-1]

tickers_v1 = tickers_v1.drop_duplicates().reset_index(drop=True)

In [145]:
len(tickers_v1)

28900

Now only a fraction of the original duplicated tickers remain.

In [146]:
duplicated = tickers_v1[tickers_v1["ticker"].duplicated(keep=False)]
print(len(duplicated["ticker"].unique()))
pd.set_option('display.max_rows', None)

duplicated.head(5)

4284


,ticker,name,active,start_date,end_date,type,cik,composite_figi
0,A,"AGILENT TECHNOLOGIES, INC",False,2003-09-10,2005-11-22,CS,1090872.0,BBG000C2V3D6
1,A,"AGILENT TECHNOLOGIES, INC",False,2005-11-28,2005-11-28,CS,1090872.0,BBG000C2V3D6
2,A,"AGILENT TECHNOLOGIES, INC",False,2005-11-30,2005-11-30,CS,1090872.0,BBG000C2V3D6
3,A,"AGILENT TECHNOLOGIES, INC",False,2005-12-02,2005-12-02,CS,1090872.0,BBG000C2V3D6
4,A,"AGILENT TECHNOLOGIES, INC",False,2005-12-06,2005-12-06,CS,1090872.0,BBG000C2V3D6


We can see that some are the same company but not back-to-back. Some are different companies or went OTC and back, these are correct. However, for a lot of stocks, the first occurence only trades for a few days. That makes no sense. If you try to download data for these dates, you will see that there exists none.

So some stocks have a 'ghost' day just before their IPO. E.g. YGF was IPO'd on 2023-03-28. But on 2023-03-24 had a entry with start_date and end_date of just one day. This is the same with VCIG, which had 2 'ghost' days on 2023-03-22 and 2023-04-06. For SXTP, the ghost days were actually two. Investigating the stocks that only have 1 day in our ticker list also shows funds (that are NOT common stocks!).

Nevertheless, if start_date is equal to end_date, it's always unusable and something is wrong. So we will first remove all tickers that only exist for one day.

In [147]:
ghost_days = tickers_v1[((tickers_v1["end_date"] - tickers_v1["start_date"]) == timedelta(days=0)) & 
                        (tickers_v1["start_date"] <= market_days[market_days.index(END_DATE) - 1])]
print(len(ghost_days))
ghost_days.head(5)

2491


,ticker,name,active,start_date,end_date,type,cik,composite_figi
1,A,"AGILENT TECHNOLOGIES, INC",False,2005-11-28,2005-11-28,CS,1090872.0,BBG000C2V3D6
2,A,"AGILENT TECHNOLOGIES, INC",False,2005-11-30,2005-11-30,CS,1090872.0,BBG000C2V3D6
3,A,"AGILENT TECHNOLOGIES, INC",False,2005-12-02,2005-12-02,CS,1090872.0,BBG000C2V3D6
4,A,"AGILENT TECHNOLOGIES, INC",False,2005-12-06,2005-12-06,CS,1090872.0,BBG000C2V3D6
5,A,"AGILENT TECHNOLOGIES, INC",False,2005-12-08,2005-12-08,CS,1090872.0,BBG000C2V3D6


In [148]:
###
tickers_v1 = tickers_v1[((tickers_v1["end_date"] - tickers_v1["start_date"]) > timedelta(days=0)) | 
                        (tickers_v1["start_date"] == market_days[market_days.index(END_DATE) - 1])]
tickers_v1 = tickers_v1.reset_index(drop=True)
len(tickers_v1)

26407

The remaining duplicates are:

In [149]:
duplicated = tickers_v1[tickers_v1["ticker"].duplicated(keep=False)]
print(len(duplicated["ticker"].unique()))

duplicated.head(5)

3616


,ticker,name,active,start_date,end_date,type,cik,composite_figi
0,A,"AGILENT TECHNOLOGIES, INC",False,2003-09-10,2005-11-22,CS,1090872.0,BBG000C2V3D6
1,A,Agilent Technologies Inc.,True,2005-12-14,2024-04-19,CS,1090872.0,BBG000C2V3D6
5,AAAP,ADVANCED ACCELERATOR APPLICATIONS S A SPONSORE...,False,2015-02-04,2015-02-05,NONE,1611787.0,BBG00K6FMBQ8
6,AAAP,Advanced Accelerator Applications S.A. America...,False,2015-11-11,2018-02-09,ADRC,1611787.0,BBG00K6FMBQ8
10,AAC,"ABLEAUCTIONS, COM, INC.",False,2003-09-10,2010-02-05,CS,1099290.0,


We need to keep in mind that the <code>start_date</code> and <code>end_date</code> may not be the start/end dates of the data. To determine the data dates, we need to loop through the ticker list and see whether the data exists. 

However, after we have downloaded our data, we can just infer it. So we will postpone this to avoid doing it twice.

In [150]:
# tickers_v1.to_csv("../data/tickers_v1_BEFORE_3.6.csv")

In [74]:
/

# 3.6 Removing incorrect classes
There are still some weird or incorrect stock classes that we have to remove. These were found by just looking through the ticker list.

These are:
- Funds. There are a LOT of them.
- Preferred stock/bonds
- Non-stocks, these have a uncapitalized letter in the ticker, such as "w" for warrants
- "Ex-distribution" or "When-issued" conditions

For the last 2 ones, it is understandable that they show up in ticker lists. However the first 2 should never.

Caution: we will remove some correct tickers. However that is better than having 100+ funds in the data...

In [75]:
###
funds = tickers_v1[tickers_v1['name'].apply(lambda s: 
        "fund" in s.lower().split() \
        or "fund," in s.lower().split() \
        or "fnd" in s.lower().split() \
        or "fd" in s.lower().split() \
        or "aberdeen" in s.lower().split() \
        or "barings," in s.lower().split() \
        or "blackrock" in s.lower().split() \
        or "bldrs" in s.lower().split() \
        or "citigroup" in s.lower().split() \
        or "direxion" in s.lower().split() \
        or "mfs" in s.lower().split() \
        or "eaton" in s.lower().split() \
        or "calamos" in s.lower().split() \
        or "nuveen" in s.lower().split() \
        or "proshares" in s.lower().split() \
        or "suisse" in s.lower().split()\
        or "ishares" in s.lower().split()\
        or "jpmorgan" in s.lower().split()\
        or "invesco" in s.lower().split()\
        or "powershares" in s.lower().split()\
        or "gabelli" in s.lower().split() \
        or "morgan" in s.lower().split() \
        or "merrill" in s.lower().split() \
        or "merill" in s.lower().split() \
        or "merr" in s.lower().split() \
        or "etf" in s.lower().split() \
        or "etn" in s.lower().split() \
        or "etv" in s.lower().split() \
        or "index" in s.lower().split() \
        or "idx" in s.lower().split() \
        or "indx" in s.lower().split() \
        or "ctf" in s.lower().split() \
        or "pwrshrs" in s.lower().split() \
        or "pwrsh" in s.lower().split() \
        or "dbx" in s.lower().split() \
        or "msdw" in s.lower().split() \
        or "structured" in s.lower().split() \
        or "tr" in s.lower().split() \
        or "ubs" in s.lower().split() \
        )

        | tickers_v1['ticker'].isin(['FSMO', 'LOR', 'CMCA', 'EMD', 'HORI', 'KEMP'])

        & ~tickers_v1['ticker'].isin(['JPM', 'EV', 'IVZ', 'IVR', 'BLK', 'TCPC', \
                'DJP', 'KMI', 'KMP', 'KMR', 'MHGC', 'MS', 'MWD', 'MR', 'MFUN', 'C', 'CS', 'CSR', 'UBS'])
        ]
print(len(funds))
funds.head(3)

3458


,ticker,name,active,start_date,end_date,type,cik,composite_figi
36,AAIT,ISHARES MSCI ALL COUNTRY ASIA INFO TECHNOLOGY ETF,False,2012-02-09,2015-08-21,NONE,1100663.0,NaN
65,AAY,MORGAN STLY **% HITS LKD ALLEGHENY TECH COM 4/...,False,2007-03-26,2008-04-21,NONE,NaN,NaN
81,ABBK.P,ABINGTON BANCORP CAP TR PFD,False,2003-09-10,2004-11-05,NONE,NaN,NaN


In [76]:
###
tickers_pct = tickers_v1[tickers_v1['name'].str.contains('%')]
print(len(tickers_pct))
tickers_pct.head(3)

1665


,ticker,name,active,start_date,end_date,type,cik,composite_figi
25,AAG.PRTC,AMER ANNTY GRP 91/4% (TOPRS),False,2004-01-30,2004-03-05,NONE,NaN,NaN
27,AAGpT,AMER ANNTY GRP 91/4% (TOPRS),False,2003-09-10,2004-01-29,NONE,NaN,NaN
35,AAIN,Arlington Asset Investment Corp. 6.000% Senior...,False,2021-07-19,2022-08-19,NONE,1209028.0,NaN


In [77]:
###
tickers_small_letter = tickers_v1[tickers_v1["ticker"].apply(lambda str: any(s.islower() for s in str))]
print(len(tickers_small_letter))
tickers_small_letter.head(3)

1960


,ticker,name,active,start_date,end_date,type,cik,composite_figi
27,AAGpT,AMER ANNTY GRP 91/4% (TOPRS),False,2003-09-10,2004-01-29,NONE,NaN,NaN
44,AANw,"The Aaron''s Company, Inc.",False,2020-11-25,2020-11-30,CS,1821393.0,BBG00WCNDCZ6
50,AAPw,ADVANCE AUTO PARTS INC WI,False,2005-09-22,2005-09-23,NONE,1158449.0,BBG000F7RCJ1


In [78]:
###
tickers_preferred = tickers_v1[tickers_v1['name'].apply(lambda s: 
               ("pf" in s.lower().split()) \
            or ("pfd" in s.lower().split()) \
            or ("pfr" in s.lower().split()) \
            or ("pref" in s.lower().split()) \
            or ("preferred" in s.lower().split()) \
            or ("exp" in s.lower().split()) \
            or ("due" in s.lower().split()) \
            or ("expiry" in s.lower().split())\
            or ("abs" in s.lower().split())\
            or ("warrant" in s.lower().split()) \
            or ("warrants" in s.lower().split()) \
            or ("crts" in s.lower().split()))
            ]
print(len(tickers_preferred))
tickers_preferred.head(3)

2298


,ticker,name,active,start_date,end_date,type,cik,composite_figi
7,AAB.WS,LEHMAN BROTHERS CURRENCY BASKET WTS EXP 2/13/2008,False,2006-08-08,2008-02-08,NONE,NaN,NaN
19,AACOU,AUSTRALIA ACQUISITION CORP UNIT SH & WT EXP,False,2010-11-12,2010-12-10,NONE,1499593.0,NaN
20,AACOU,AUSTRALIA ACQUISITION CORP UNIT SH & WT EXP,False,2010-12-15,2012-10-15,NONE,1499593.0,NaN


In [79]:
###
when_issued_or_ex_distr = tickers_v1[tickers_v1['name'].apply(lambda s: 
               ("when" in s.lower().split()) \
            or ("issued" in s.lower().split()) \
            or ("when-issued" in s.lower().split()) \
            or ("ex-distribution" in s.lower().split()) \
            or ("w.i." in s.lower().split()) \
            or ("wts" in s.lower().split())   )  ]
print(len(when_issued_or_ex_distr))
when_issued_or_ex_distr.head(3)

1012


,ticker,name,active,start_date,end_date,type,cik,composite_figi
7,AAB.WS,LEHMAN BROTHERS CURRENCY BASKET WTS EXP 2/13/2008,False,2006-08-08,2008-02-08,NONE,NaN,NaN
85,ABBVw,ABBVIE INC COM STK (DE) W.I.,False,2012-12-10,2012-12-31,CS,1551152.0,BBG0025Y4RY4
100,ABDw,ACCO BRANDS CORPORATION W.I.,False,2005-08-09,2005-08-16,CS,712034.0,BBG000J06K07


In [80]:
###
tickers_suffix = tickers_v1[
        tickers_v1["ticker"].str.contains("\.WD") |
        tickers_v1["ticker"].str.contains("\.W") |
        tickers_v1["ticker"].str.contains("\.Z") |
        tickers_v1["ticker"].str.contains("\.V") |
        tickers_v1["ticker"].str.contains("\.U") |
        tickers_v1["ticker"].str.contains("\.PRTC") |
        tickers_v1["ticker"].str.contains("\.RTS")
        ]
print(len(tickers_suffix))
tickers_suffix.head(3)

850


,ticker,name,active,start_date,end_date,type,cik,composite_figi
7,AAB.WS,LEHMAN BROTHERS CURRENCY BASKET WTS EXP 2/13/2008,False,2006-08-08,2008-02-08,NONE,NaN,NaN
25,AAG.PRTC,AMER ANNTY GRP 91/4% (TOPRS),False,2004-01-30,2004-03-05,NONE,NaN,NaN
103,ABEO.W,ABEONA THERAPEUTICS INC WT PUR COM,False,2015-06-22,2016-10-24,NONE,318306.0,NaN


In [81]:
###
indices_to_remove = set().union(*[set(funds.index), 
                                  set(tickers_pct.index),
                                  set(tickers_small_letter.index),
                                  set(tickers_preferred.index),
                                  set(when_issued_or_ex_distr.index),
                                  set(tickers_suffix.index),
                                  ])

In [82]:
###
print(len(tickers_v1))
tickers_v1 = tickers_v1.drop(index=indices_to_remove)
print(len(tickers_v1))

26407
19022


In [83]:
###
tickers_v1 = tickers_v1.reset_index(drop=True)
tickers_ADR = tickers_v1[(tickers_v1['name'].apply(lambda s: 
               ("n.v." in s.lower().split()) \
            or ("sa" in s.lower().split()) \
            or ("adr" in s.lower().split()) \
            or ("ads" in s.lower().split()) \
            or ("ltd" in s.lower().split()) \
            or ("plc" in s.lower().split()) \
            or ("depositary" in s.lower().split()) \
            or ("sponsored" in s.lower().split()) \
                )) & (tickers_v1['type'] == 'NONE')]

tickers_v1.iloc[tickers_ADR.index, 5] = 'ADRC'
tickers_v1.loc[tickers_v1['type'] == 'NONE', 'type'] = 'CS'

In [84]:
###
tickers_v1["ID"] = tickers_v1["ticker"] + '-' + tickers_v1["start_date"].astype(str)

Old code to check for misclassifications by checking the ticker type before start end after end date. However this misclassified some extra tickers. Therefore I don't use it.

In [85]:
# client.get_ticker_details(ticker="ACP", date = "2019-04-02").type

In [86]:
# client.get_ticker_details(ticker="ACP", date = "2019-04-01").type

In [87]:
# ###
# IDs_to_remove = []
# for index, row in tickers_v1.copy().iterrows():
#     try:
#         # The get_ticker_details is extremely slow.
#         type_before_start = client.get_ticker_details(ticker=row['ticker'], date = last_trading_date_before(row['start_date']) ).type
#     except Exception as e:
#         # If there is no information. we will keep it.
#         type_before_start = 'CS'

#     try: 
#         type_after_end = client.get_ticker_details(ticker=row['ticker'], date = first_trading_date_after(row['end_date']) ).type
#     except Exception as e:
#         # If there is no information. we will keep it.
#         type_after_end = 'CS'

#     # Apparently an ordinary share is the same as a common stock... Also do not forget ADRCs!
#     if type_before_start not in ['CS', 'ADRC', 'OS', None] or type_after_end not in ['CS', 'ADRC', 'OS', None]:
#         IDs_to_remove.append(row['ID'])
#         print(row['ID'])

In [88]:
# ###
# incorrect_classified = tickers_v1[tickers_v1['ID'].isin(IDs_to_remove)]
# print(len(incorrect_classified))
# incorrect_classified.to_csv("../data/incorrect_classified.csv")
# incorrect_classified.tail(3)[['ticker', 'name', 'active', 'start_date', 'end_date']]

In [89]:
# ###
# to_remove = pd.read_csv("../data/incorrect_classified.csv", index_col=0)
# IDs_to_remove = list(to_remove.index)

# print(len(tickers_v1))
# tickers_v1 = tickers_v1[~tickers_v1['ID'].isin(IDs_to_remove)]
# tickers_v1.reset_index(inplace=True, drop=True)
# print(len(tickers_v1))

Finally, we save the merged and cleaned ticker list to <code>tickers_v2.csv</code>.

In [90]:
###
print(f"Total tickers: {len(tickers_v1)}")
print(f"Unique tickers: {len(tickers_v1['ticker'].unique())}")

tickers_v1 = tickers_v1.reset_index(drop=True)
tickers_v1 = tickers_v1[["ID", "ticker", "name", "active", "start_date", "end_date", "type", "cik", "composite_figi"]]
tickers_v1.to_csv("../data/tickers_v2.csv")

Total tickers: 19022
Unique tickers: 16188


# 3.7 Updates
1. Run the first cell below to create a backup of <code>tickers_v3</code>. Later when we download data, we will compare the old to the new ticker list to determine which stocks and dates to update, instead of downloading everything.
1. Update END_DATE.
2. 3.1: Run the 3 cells with ###. This updates the folder of ticker lists.
3. Run the below cells to get the new <code>tickers_v1</code>.
4. 3.5, 3.6: Run everything (merge, remove incorrect classes) to get <code>tickers_v2</code>.

In [ ]:
tickers_v3 = get_tickers(3)
tickers_v3.to_csv("../data/tickers_v3_old.csv")

This is the tickers loop, however the first ticker list is tickers_v1 instead of the ticker list of the first day. This is much faster than redoing everything.

In [ ]:
from tickers import get_tickers
market_days = get_market_dates()

tickers_v1 = get_tickers(1)
CURRENT_END_DATE = tickers_v1['end_date'].max()
END_DATE = last_trading_date_before_equal(END_DATE)

market_days = get_market_dates()

for day in market_days:
    if day == CURRENT_END_DATE:
        # At the start, our ticker list is our current tickers_v1
        our_tickers = tickers_v1
        our_tickers.loc[our_tickers['active'], 'end_date'] = np.NaN
        our_tickers['cik'] = our_tickers['cik'].replace('', np.nan)
        our_tickers['cik'] = our_tickers['cik'].astype(float)

    elif day > CURRENT_END_DATE and day <= END_DATE:
        # Get new ticker list to update ours
        tickers_day_i = pd.read_csv(
            POLYGON_DATA_PATH + f"raw/tickers/{day.isoformat()}.csv",
            index_col=0,
            keep_default_na=False,
            na_values=['#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN', '-NaN', '-nan', '1.#IND', \
            '1.#QNAN', '<NA>', 'N/A', 'NULL', 'NaN', 'None', 'n/a', 'nan', 'null']
        )
        # Sometimes a cik can be empty. This prevents the merge function from working.
        tickers_day_i['cik'] = tickers_day_i['cik'].replace('', np.nan)
        tickers_day_i['cik'] = tickers_day_i['cik'].astype(float)

        tickers_day_i.sort_values(['last_updated_utc'], inplace=True)

        tickers_day_i = tickers_day_i[["ticker", "name", "active", "cik", "composite_figi", "type"]]
        tickers_day_i = tickers_day_i[tickers_day_i["active"]]
        tickers_day_i.reset_index(inplace=True, drop=True)

        # Remove duplicates (very rare, but sometimes this happens due to duplicates in polygons ticker lists)
        duplicated = tickers_day_i[tickers_day_i["ticker"].duplicated(keep=False)]
        indices_to_remove = duplicated["ticker"].duplicated(keep='last')
        tickers_day_i.drop(list(indices_to_remove[indices_to_remove].index), inplace=True)
        tickers_day_i.reset_index(drop=True, inplace=True)

        # Preliminary check: no duplicates
        if our_tickers.duplicated().any():
            raise Exception("There are duplicates!")

        if tickers_day_i.duplicated().any():
            raise Exception("There are duplicates!")

        # DELISTINGS
        indicator_delisted = our_tickers[["ticker", "name", "active", "cik", "composite_figi", "type"]].\
            merge(tickers_day_i[["ticker", "name", "active", "cik", "composite_figi", "type"]], \
                on=["ticker", "name", "active", "cik", "composite_figi", "type"], how='left', indicator=True)

        indicator_delisted['_merge'] = np.where(our_tickers["active"], indicator_delisted['_merge'], "both") # ERROR FIX: If in our ticker list we have already set it inactive, it should not be added to the list of delisted stocks again. By setting _merge to "both" we skip the already inactive stocks.

        indicator_delisted = indicator_delisted["_merge"] # Only get the indicator
        delisted_tickers = our_tickers[indicator_delisted == "left_only"]

        # NEW LISTINGS
        indicator_new = tickers_day_i[["ticker", "name", "active", "cik", "composite_figi", "type"]].\
            merge(our_tickers[["ticker", "name", "active", "cik", "composite_figi", "type"]], \
                on=["ticker", "name", "active", "cik", "composite_figi", "type"], 
                        how='left', indicator=True)
        indicator_new = indicator_new["_merge"]
        new_tickers = tickers_day_i[indicator_new == "left_only"]

        # PROCESS DELISTINGS
        previous_day = market_days[market_days.index(day) - 1] # Getting previous trading day
        our_tickers.loc[indicator_delisted == "left_only", "end_date"] = previous_day
        our_tickers.loc[indicator_delisted == "left_only", "active"] = False
        
        # PROCESS NEW LISTINGS
        our_tickers = pd.concat([our_tickers, new_tickers])
        our_tickers.reset_index(inplace=True, drop=True)
        our_tickers['start_date'].fillna(value=day, inplace=True)

        # Final checks
        if our_tickers[["ticker", "name", "active", "type", "start_date"]].isnull().values.any():
            #null_data = our_tickers[our_tickers[["ticker", "name", "active", "type", "start_date"]].isnull().any(axis=1)]
            raise Exception("There are missing values.")
        
        print(f'{day.isoformat()}: Amount of stocks {len(our_tickers)}')
        
        # Finalize
        if day == END_DATE:
            our_tickers["end_date"].fillna(value=END_DATE, inplace=True)
            our_tickers = our_tickers.sort_values(by=["ticker", "end_date"]).reset_index(drop=True)
            our_tickers[["ticker", "name", "active", "start_date", "end_date", "type", "cik", "composite_figi"]].\
                to_csv("../data/tickers_v1.csv")